In [ ]:
import os
import time
import pickle
import requests
import itertools
import optuna
import csv
from scipy.stats import mannwhitneyu
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
tqdm.pandas()

from rdkit import RDLogger
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, rdMolTransforms
from rdkit.Chem.MolStandardize import rdMolStandardize
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.molSize = 300,300
 
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score, confusion_matrix, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold
import networkx as nx

import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import SAGEConv, GCNConv, GATConv, GINConv
from torch_geometric.data import Data, Dataset, InMemoryDataset#, DataLoader depreciated, use below
from torch_geometric.loader import DataLoader
from torch.utils.data import Subset

In [ ]:
class GNNModel(torch.nn.Module):
    def __init__(self, conv_type, hidden_channels, num_features, batch_norm=False, weight_init=True, dropout_rate=0.0):
        super(GNNModel, self).__init__()
        self.conv1 = conv_type(num_features, hidden_channels*2)
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels*2) if batch_norm else None #BatchNorm1D -> GraphNorm
        
        self.conv2 = conv_type(hidden_channels*2, hidden_channels)
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels) if batch_norm else None
        
        self.conv3 = conv_type(hidden_channels, hidden_channels//2)
        
        self.dropout = torch.nn.Dropout(dropout_rate) if dropout_rate > 0 else None
        self.lin = torch.nn.Linear(hidden_channels//2, 1)
        if weight_init:
            self.apply(self._init_weights)

    def _init_weights(self, module):
        """Apply Xavier initialisation to the weights of a given module."""
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, torch_geometric.nn.MessagePassing):
            # Initialising weights of the MessagePassing (convolution) layers
            for param in module.parameters():
                if param.dim() > 1:  # Only initialise weights, not biases
                    torch.nn.init.xavier_uniform_(param)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        if self.bn1:
            x = self.bn1(x)
        x = F.relu(x)
        #if self.dropout:
        #    x = self.dropout(x)
            
        x = self.conv2(x, edge_index)
        if self.bn2:
            x = self.bn2(x)
        x = F.relu(x)
        #if self.dropout:
        #    x = self.dropout(x)
            
        x = self.conv3(x, edge_index)
        #if self.bn3:
        #    x = self.bn3(x)
        x = F.relu(x)
        if self.dropout:
            x = self.dropout(x)
            
        x = torch_geometric.nn.global_mean_pool(x, batch)
        x = self.lin(x)
        return x

class GINModel(torch.nn.Module):
    def __init__(self, hidden_channels, num_features, batch_norm=False, weight_init=True, dropout_rate=0.0):
        super(GINModel, self).__init__()
        # Define MLP for GINConv
        def mlp(input_dim, output_dim):
            return torch.nn.Sequential(
                torch.nn.Linear(input_dim, output_dim),
                torch.nn.ReLU(),
                torch.nn.Linear(output_dim, output_dim)
            )

        self.conv1 = torch_geometric.nn.GINConv(mlp(num_features, hidden_channels*2))
        self.bn1 = torch.nn.BatchNorm1d(hidden_channels*2) if batch_norm else None
        
        self.conv2 = torch_geometric.nn.GINConv(mlp(hidden_channels*2, hidden_channels))
        self.bn2 = torch.nn.BatchNorm1d(hidden_channels) if batch_norm else None
        
        self.conv3 = torch_geometric.nn.GINConv(mlp(hidden_channels, hidden_channels//2))
        
        self.dropout = torch.nn.Dropout(dropout_rate) if dropout_rate > 0 else None
        self.lin = torch.nn.Linear(hidden_channels//2, 1)

        # Apply Xavier initialisation
        if weight_init:
            self.apply(self._init_weights)

    def _init_weights(self, module):
        """Apply Xavier initialisation to the weights of a given module."""
        if isinstance(module, torch.nn.Linear):
            torch.nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, GINConv):
            # Initialising weights of the GINConv's MLPs
            for layer in module.nn:
                if isinstance(layer, torch.nn.Linear):
                    torch.nn.init.xavier_uniform_(layer.weight)
                    if layer.bias is not None:
                        torch.nn.init.zeros_(layer.bias)

    def forward(self, x, edge_index, batch):
        x = self.conv1(x, edge_index)
        if self.bn1:
            x = self.bn1(x)
            x = F.relu(x) # Only if batch normalisation is performed
        #if self.dropout:
        #    x = self.dropout(x)
            
        x = self.conv2(x, edge_index)
        if self.bn2:
            x = self.bn2(x)
            x = F.relu(x)
        #if self.dropout:
        #    x = self.dropout(x)
            
        x = self.conv3(x, edge_index)
        #if self.bn3:
        #    x = self.bn3(x)
        #    x = F.relu(x)
        if self.dropout:
            x = self.dropout(x)
            
        x = torch_geometric.nn.global_mean_pool(x, batch)
        x = self.lin(x)
        return x

In [ ]:
def train(model, loader, optimiser, criterion, device, threshold = 0.5): # added
    model.train()
    total_loss = 0
    correct = 0
    for data in loader:
        data = data.to(device)
        optimiser.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        loss = criterion(out, data.y.view(-1, 1).float())
        loss.backward()
        optimiser.step()
        total_loss += loss.item()
        pred = (out > threshold).float() # added
        correct += pred.eq(data.y.view(-1, 1)).sum().item()
    return total_loss / len(loader), correct / len(loader.dataset)

def evaluate(model, loader, criterion, device, threshold = 0.5): # added
    model.eval()
    total_loss = 0
    correct = 0
    preds, labels = [], []
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            out = model(data.x, data.edge_index, data.batch)
            loss = criterion(out, data.y.view(-1, 1).float())
            total_loss += loss.item()
            pred = (out > threshold).float() # added
            correct += pred.eq(data.y.view(-1, 1)).sum().item()
            preds.append(out)
            labels.append(data.y.view(-1, 1))
    preds = torch.cat(preds, dim=0)
    labels = torch.cat(labels, dim=0)
    return total_loss / len(loader), correct / len(loader.dataset), roc_auc_score(labels.cpu(), preds.cpu()), f1_score(labels.cpu(), (preds > threshold).float().cpu()), matthews_corrcoef(labels.cpu(), (preds > threshold).float().cpu()) # added # modified for MCC

class EarlyStopping:
    def __init__(self, patience=10, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_score = None
        self.early_stop = False
        self.counter = 0

    def __call__(self, val_score, model):
        score = -val_score
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_score, model)
        elif score < self.best_score - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_score, model)
            self.counter = 0

    def save_checkpoint(self, val_score, model):
        self.best_model = model

In [ ]:
def get_fold_indices(pytorch_custom_dataset, seed=42):
    num_samples = len(pytorch_custom_dataset)
    indices = np.arange(num_samples)
    targets = [data.y.item() for data in pytorch_custom_dataset]

    skf_outer = StratifiedKFold(n_splits=4, shuffle=True, random_state=seed)
    outer_folds = []

    for train_indices, test_indices in skf_outer.split(indices, targets):
        train_targets = np.array(targets)[train_indices]
        skf_inner = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        inner_folds = []
        for inner_train_idx, val_idx in skf_inner.split(np.arange(len(train_indices)), train_targets):
            inner_train_indices = train_indices[inner_train_idx]
            val_indices = train_indices[val_idx]
            inner_folds.append((inner_train_indices, val_indices))
        outer_folds.append((train_indices, test_indices, inner_folds)) 
    return outer_folds

def objective(hyperparameters, model_class, conv_type, pytorch_custom_dataset, device, results_dir, model_name, outer_fold, inner_fold, fold_indices, use_early_stopping):
    # Extract hyperparameters
    learning_rate = hyperparameters['learning_rate']
    hidden_channels = hyperparameters['hidden_channels']
    batch_size = hyperparameters['batch_size']
    dropout_rate = hyperparameters['dropout_rate']
    num_epochs = 100  # Fixed number of epochs

    # Set seed for reproducibility
    torch.manual_seed(1)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(1)

    # Get the indices for the current fold
    train_indices, test_indices, inner_folds = fold_indices[outer_fold]
    inner_train_indices, val_indices = inner_folds[inner_fold]

    # Create datasets
    train_dataset = Subset(pytorch_custom_dataset, inner_train_indices)
    val_dataset = Subset(pytorch_custom_dataset, val_indices)
    test_dataset = Subset(pytorch_custom_dataset, test_indices)

    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    num_features = pytorch_custom_dataset[0].num_features  # Assuming dataset has num_features attribute

    # Initialize the model
    if model_class == GINModel:
        model = model_class(
            hidden_channels=hidden_channels,
            num_features=num_features,
            batch_norm=False,
            weight_init=True,
            dropout_rate=dropout_rate
        ).to(device)
    else:
        model = model_class(
            conv_type=conv_type,
            hidden_channels=hidden_channels,
            num_features=num_features,
            batch_norm=False,
            weight_init=True,
            dropout_rate=dropout_rate
        ).to(device)

    optimiser = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.BCEWithLogitsLoss()

    # Training loop with early stop function
    early_stopping = EarlyStopping(patience = 10, delta=0)
    
    for epoch in range(1, num_epochs + 1):
        train_loss, train_acc = train(model, train_loader, optimiser, criterion, device)
        val_loss, val_acc, val_auc, _, _ = evaluate(model, val_loader, criterion, device)
        if use_early_stopping:
            early_stopping(val_auc, model)
            if early_stopping.early_stop:
                print(f"Early stopping at epoch {epoch}")
                break
                
    best_model = early_stopping.best_model if use_early_stopping else model
    val_loss, val_acc, val_auc, _, _ = evaluate(best_model, val_loader, criterion, device)
    
    return val_auc

def grid_search(model_class, conv_type, pytorch_custom_dataset, results_dir, model_name, use_early_stopping):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    os.makedirs(results_dir, exist_ok=True)

    # Get the fold indices once
    fold_indices = get_fold_indices(pytorch_custom_dataset, seed=42)

    # Define hyperparameter grid
    batch_size_list = [32, 64, 128]
    hidden_channels_list = [64, 128, 256]
    learning_rate_list = [0.001, 0.0001, 0.00001]
    dropout_rate_list = [0.1, 0.2, 0.3, 0.4, 0.5]

    hyperparameter_grid = list(itertools.product(
        batch_size_list,
        hidden_channels_list,
        learning_rate_list,
        dropout_rate_list
    ))

    # List to store results
    results = []

    for outer_fold in range(4):
        print(f"Outer Fold {outer_fold + 1}/4")

        for inner_fold in range(5):
            print(f"  Inner Fold {inner_fold + 1}/5")

            for batch_size, hidden_channels, learning_rate, dropout_rate in hyperparameter_grid:
                # Create a dictionary of hyperparameters
                hyperparameters_dict = {
                    'batch_size': batch_size,
                    'hidden_channels': hidden_channels,
                    'learning_rate': learning_rate,
                    'dropout_rate': dropout_rate
                }

                # Call the objective function
                val_auc = objective(
                    hyperparameters=hyperparameters_dict,
                    model_class=model_class,
                    conv_type=conv_type,
                    pytorch_custom_dataset=pytorch_custom_dataset,
                    device=device,
                    results_dir=results_dir,
                    model_name=model_name,
                    outer_fold=outer_fold,
                    inner_fold=inner_fold,
                    fold_indices=fold_indices,
                    use_early_stopping = use_early_stopping
                )
                print(f"Testing hyperparameters: bs={batch_size}, hc={hidden_channels}, lr={learning_rate}, dr={dropout_rate} / val_auc: {val_auc}")
                
                # Store the results
                result = {
                    'model_name': model_name,
                    'outer_fold': outer_fold,
                    'inner_fold': inner_fold,
                    'batch_size': batch_size,
                    'hidden_channels': hidden_channels,
                    'learning_rate': learning_rate,
                    'dropout_rate': dropout_rate,
                    'val_auc': val_auc
                }

                results.append(result)

    # Return all results
    return results

def run_grid_search_for_all_models(models, pytorch_custom_dataset, results_dir, use_early_stopping):
    all_results = []

    for name, model_class in models.items():
        print(f"Running Grid Search for {name} model...")

        # Define the convolution type (conv_type) based on the model name
        if "GCN" in name:
            conv_type = GCNConv
        elif "GAT" in name:
            conv_type = GATConv
        elif "GraphSAGE" in name:
            conv_type = SAGEConv
        else:
            conv_type = None  # For GINModel, no conv_type is needed

        # Run grid search for this model
        results = grid_search(
            model_class=model_class,
            conv_type=conv_type,
            pytorch_custom_dataset=pytorch_custom_dataset,
            results_dir=results_dir,
            model_name=name,
            use_early_stopping = use_early_stopping
        )

        # Add results to all_results
        all_results.extend(results)

    return all_results


#### Drug-induced liver injury

In [ ]:
# Generated from Final_feature_enginnering.ipynb
df_dili_filtered_cleaned = pd.read_csv('chem_data/DILI/DILIst_standardised_cleaned.csv', index_col = 0)

dlst_min = MoleculeDataset_minimal(dataframe = df_dili_filtered_cleaned,
                              smiles_col = 'smiles',
                             label_col = 'label',
                             root = 'custom_data/minimal_feature/dlst_min'
                            )
dlst_cus_2_std = MoleculeDataset_updated(dataframe = df_dili_filtered_cleaned,
                              smiles_col = 'smiles_std',
                             label_col = 'label',
                             root = 'custom_data/DILI/cus2_std/'
                            )

In [ ]:
# dlst_cus_2_std
models = {
    'GCN_Optimised': GNNModel,
    'GAT_Optimised': GNNModel,
    'GraphSAGE_Optimised': GNNModel,
    'GIN_Optimised': GINModel
}

results_dir = './results/grid_search/dlst'

# Run Grid Search
all_results = run_grid_search_for_all_models(
    models,
    dlst_cus_2_std,
    results_dir,
    use_early_stopping = True
)


results_df = pd.DataFrame(all_results)
results_df.to_csv(os.path.join(results_dir, 'grid_search_results.csv'), index=False)

with open(os.path.join(results_dir, 'grid_search_results.pkl'), 'wb') as f:
    pickle.dump(results_df, f)

In [ ]:
# dlst_min
models = {
    'GCN_Optimised': GNNModel,
    'GAT_Optimised': GNNModel,
    'GraphSAGE_Optimised': GNNModel,
    'GIN_Optimised': GINModel
}

results_dir = './results/grid_search/dlst_min'

# Run Grid Search
all_results = run_grid_search_for_all_models(
    models,
    dlst_min,
    results_dir,
    use_early_stopping = True
)

# Create DataFrame 
results_df = pd.DataFrame(all_results)

# Save DataFrame
results_df.to_csv(os.path.join(results_dir, 'grid_search_results.csv'), index=False)

# Optionally, save the DataFrame as a pickle file
with open(os.path.join(results_dir, 'grid_search_results.pkl'), 'wb') as f:
    pickle.dump(results_df, f)

#### Blood-Brain Barrier Permeability

In [ ]:
df_bbbp_filtered_cleaned = pd.read_csv('chem_data/bbbp_standardised_cleaned.csv', index_col = 0)

bbbp_min = MoleculeDataset_minimal(dataframe = df_bbbp_filtered_cleaned,
                              smiles_col = 'smiles',
                             label_col = 'label',
                             root = 'custom_data/minimal_feature/bbbp_min'
                            )
bbbp_cus_2_std = MoleculeDataset_updated(dataframe = df_bbbp_filtered_cleaned,
                              smiles_col = 'smiles_std',
                             label_col = 'label',
                             root = 'custom_data/bbbp/cus2_std/'
                            )

In [ ]:
# bbbp_cus_2_std
models = {
    'GCN_Optimised': GNNModel,
    'GAT_Optimised': GNNModel,
    'GraphSAGE_Optimised': GNNModel,
    'GIN_Optimised': GINModel
}

results_dir = './results/grid_search/bbbp'

all_results = run_grid_search_for_all_models(
    models,
    bbbp_cus_2_std,
    results_dir,
    use_early_stopping = True
)

results_df = pd.DataFrame(all_results)
results_df.to_csv(os.path.join(results_dir, 'grid_search_results.csv'), index=False)

with open(os.path.join(results_dir, 'grid_search_results.pkl'), 'wb') as f:
    pickle.dump(results_df, f)

In [ ]:
# bbbp_min
models = {
    'GCN_Optimised': GNNModel,
    'GAT_Optimised': GNNModel,
    'GraphSAGE_Optimised': GNNModel,
    'GIN_Optimised': GINModel
}

results_dir = './results/grid_search/bbbp_min'

all_results = run_grid_search_for_all_models(
    models,
    bbbp_min,
    results_dir,
    use_early_stopping = True
)

results_df = pd.DataFrame(all_results)
results_df.to_csv(os.path.join(results_dir, 'grid_search_results.csv'), index=False)

with open(os.path.join(results_dir, 'grid_search_results.pkl'), 'wb') as f:
    pickle.dump(results_df, f)

#### BACE Activity

In [ ]:
df_bace_filtered_cleaned = pd.read_csv('chem_data/bace_standardised_cleaned.csv', index_col = 0)

bace_min = MoleculeDataset_minimal(dataframe = df_bace_filtered_cleaned,
                              smiles_col = 'smiles',
                             label_col = 'label',
                             root = 'custom_data/minimal_feature/bace_min'
                            )
bace_cus_2_std = MoleculeDataset_updated(dataframe = df_bace_filtered_cleaned,
                              smiles_col = 'smiles_std',
                             label_col = 'label',
                             root = 'custom_data/bace/cus2_std/'

In [ ]:
# bace_cus_2_std
models = {
    'GCN_Optimised': GNNModel,
    'GAT_Optimised': GNNModel,
    'GraphSAGE_Optimised': GNNModel,
    'GIN_Optimised': GINModel
}

results_dir = './results/grid_search/bace'

all_results = run_grid_search_for_all_models(
    models,
    bace_cus_2_std,
    results_dir,
    use_early_stopping = True
)

results_df = pd.DataFrame(all_results)
results_df.to_csv(os.path.join(results_dir, 'grid_search_results.csv'), index=False)

with open(os.path.join(results_dir, 'grid_search_results.pkl'), 'wb') as f:
    pickle.dump(results_df, f)

In [ ]:
# bace_min
models = {
    'GCN_Optimised': GNNModel,
    'GAT_Optimised': GNNModel,
    'GraphSAGE_Optimised': GNNModel,
    'GIN_Optimised': GINModel
}

results_dir = './results/grid_search/bace_min'

all_results = run_grid_search_for_all_models(
    models,
    bace_min,
    results_dir,
    use_early_stopping = True
)

results_df = pd.DataFrame(all_results)
results_df.to_csv(os.path.join(results_dir, 'grid_search_results.csv'), index=False)

with open(os.path.join(results_dir, 'grid_search_results.pkl'), 'wb') as f:
    pickle.dump(results_df, f)